In [1]:
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.set_num_threads(8)

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR100",
            'random_class_idx': False,

            # benchmark
            'seed': seed,
            'num_tasks': 10,
            'epochs_per_task': 100,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 32,
            'batch_size_train': 256,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 10,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.1,
            'learning_rate_decay_epoch': [30, 50, 70, 90],
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.001,
            'lambda': .01,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=CIFAR100/seed=10_epoch=100_lr=0.1_alpha=0.001_tau=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                    per_task_memory_examples=params['per_task_memory_examples'],
                    per_task_examples = params['per_task_examples'],
                    random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)



[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Files already downloaded and verified
Files already downloaded and verified


In [3]:
torch.get_num_threads()

8

In [4]:
num_classes

100

In [5]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2
from backbones import MLP2Layers2

from backbones import ResNet18Small2
backbone = ResNet18Small2(
    input_dim=input_dim, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [6]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [7]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.35900000000000004, 'loss': 0.00730016565322876, 'std': 0.2651961538182634, 'EER': -1}
[2] Eval metrics for task 1 >> {'accuracy': 0.434, 'loss': 0.006261192560195923, 'std': 0.19022092419079453, 'EER': -1}
[3] Eval metrics for task 1 >> {'accuracy': 0.491, 'loss': 0.005781925439834594, 'std': 0.2241628872048181, 'EER': -1}
[4] Eval metrics for task 1 >> {'accuracy': 0.5429999999999999, 'loss': 0.005510348200798034, 'std': 0.16745447142432474, 'EER': -1}
[5] Eval metrics for task 1 >> {'accuracy': 0.557, 'loss': 0.005111460208892822, 'std': 0.11419719786404568, 'EER': -1}
[6] Eval metrics for task 1 >> {'accuracy': 0.5509999999999999, 'loss': 0.005304158449172974, 'std': 0.16676030702778163, 'EER': -1}
[7] Eval metrics for task 1 >> {'accuracy': 0.575, 'loss': 0.0049851593971252444, 'std': 0.14108508071373102, 'EER': -1}
[8] Eval metrics for task 1 >> {'accuracy': 0.623, 'loss': 0.0

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [8]:
metric_manager_callback.meters['EER'].get_eer()

[0.0, 0.1865, 0.38422222222222224, 0.315125, 0.23364000000000001]

In [9]:
metric_manager_callback.meters['std'].get_std()

[0.009000000000000008,
 0.2092146027408221,
 0.4080785940096453,
 0.36503381692659653,
 0.30184141531605635]

In [10]:
metric_manager_callback.meters['accuracy'].get_data()

array([[0.977, 0.   , 0.   , 0.   , 0.   ],
       [0.259, 0.632, 0.   , 0.   , 0.   ],
       [0.003, 0.005, 0.868, 0.   , 0.   ],
       [0.004, 0.036, 0.01 , 0.857, 0.   ],
       [0.032, 0.203, 0.026, 0.173, 0.838]])

In [11]:
metric_manager_callback.meters['accuracy'].compute_overall()

0.4391633333333333

In [12]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"EER:{np.mean(metric_manager_callback.meters['EER'].compute_overall())}")
print(f"std:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")


accuracy:0.4391633333333333
EER:0.22389744444444445
std:0.25863368579862406
